# Library import

In [ ]:
git clone https://github.com/ultralytics/yolov5  # clone
cd yolov5
pip install -r requirements.txt  # install

In [ ]:
import os

import yaml
from sklearn.model_selection import GroupKFold

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch.nn as nn


## Path

In [ ]:
# path per la generazione del file .yalm da utilizzare per il finetuning 
cwd = '/kaggle/working/'

train_file = os.path.join(cwd, 'train.txt')
val_file = os.path.join(cwd, 'val.txt')
yaml_file = os.path.join(cwd, 'data_dictionary_yaml.yaml')


# path per la gestione del modello e dell'addestramento
model_path = "yolov5s.pt"


# Dataloader

## Scrittura di un dataset yaml che definisce i percorsi treno/val e i nomi delle classi

In [ ]:
# Creazione del file 'train.txt'
with open(train_file, 'w') as f:
    for path in train_df.image_path.tolist():
        f.write(path + '\n')

# Creazione del file 'val.txt'
with open(val_file, 'w') as f:
    for path in valid_df.image_path.tolist():
        f.write(path + '\n')

# Dati YAML
data = dict(
    path='/kaggle/working',
    train=train_file,
    val=val_file,
    nc=1,
    names=['cots'],
)

# Scrittura del file 'gbr.yaml'
with open(yaml_file, 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)

# Lettura e stampa del contenuto del file 'gbr.yaml'
with open(yaml_file, 'r') as f:
    print('\nyaml:')
    print(f.read())

# Network

In [ ]:
class YoloModel(nn.Module):
    """
    Classe YOLOv5 per definire il modello e la funzione di forward.
    """
    def __init__(self, model_path="yolov5s.pt"):
        """
        Inizializza il modello YOLOv5.

        Args:
            model_path (str): Percorso ai pesi pre-addestrati YOLOv5.
        """
        super(YoloModel, self).__init__()
        self.model_path = model_path
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    def forward(self, images):
        """
        Esegue la predizione sul batch di immagini.

        Args:
            images (torch.Tensor): Batch di immagini di input.

        Returns:
            torch.Tensor: Risultati delle predizioni.
        """
        return self.model(images)


## per addestrare il modello
### python train.py --img 640 --epochs 3 --data dataset.yaml --weights yolov5s.pt

In [ ]:
class Trainer:
    """
    Classe per addestrare un modello YOLOv5.
    """
    def __init__(self, model, dataset_yaml, img_size=640, batch_size=16, epochs=50, cache="ram"):
        """
        Inizializza il Trainer per YOLOv5.

        Args:
            model (YoloModel): Istanza del modello YOLOv5.
            dataset_yaml (str): Percorso al file di configurazione del dataset.
            img_size (int): Dimensione delle immagini di input.
            batch_size (int): Dimensione del batch per l'addestramento.
            epochs (int): Numero di epoche.
            cache (str): Tipo di caching ('ram' o 'disk').
        """
        self.model = model
        self.dataset_yaml = dataset_yaml
        self.img_size = img_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.cache = cache

    def train(self): 
        """
        Avvia l'addestramento del modello utilizzando YOLOv5.
        """
        command = (
            f"python train.py --img {self.img_size} --batch-size {self.batch_size} "
            f"--epochs {self.epochs} --optimizer {OPTMIZER} "
            f"--data {self.dataset_yaml} "
            f"--weights {self.model.model_path} --cache {self.cache}"
        )
        os.system(command)
        print("Addestramento completato.")

    def validate(self, weights_path=None):
        """
        Valida il modello sui dati di test.

        Args:
            weights_path (str, optional): Percorso ai pesi addestrati. Se None, usa i pesi attuali del modello.
        """
        weights = weights_path if weights_path else self.model.model_path
        command = f"python val.py --data {self.dataset_yaml} --weights {weights} --img {self.img_size}"
        os.system(command)
        print("Validazione completata.")
